# HyperSeg: Patch-wise Hypernetwork for Real-time Semantic Segmentation  
- paper review

## Method  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/HyperSeg/figures/figure2.png?raw=true">  

**Overview**  
- HyperSeg는 encoder와 subsequent blocks를 사용하며 context head와 weight mapper를 이용한다  
- 3개의 sub-networks가 존재한다  
    - 1. backbone
    - 2. context head  
    - 3. decoder consists of multiple meta blocks  
    - 이때 각 meta block은 additional weight mapping network component $w_i$를 포함하고 있다  
    
**Information flow**  
- 위 3개 networks의 weight는 $\theta_b$, $\theta_h$, $\theta_w$로 표현하며 decoder meta block $m_i$의 weight $\theta^{m_i}$ sms inference 과정에서 dynamically하게 예측되는 파라미터다  
- encoder의 backbone은 input image $I\in\mathcal{R}^{3\times H\times W}$을 feature map $F_i\in\mathcal{R}^{C_i\times \frac{H}{2^i}\times\frac{W}{2^i}}$으로 매핑을 한다  
- context head는 backbone의 last feature map $\mathcal{R}^{C_n\times \frac{H}{2^n}\times\frac{W}{2^n}}$을 signal $\phi$ $\mathcal{R}^{C_n\times \frac{H}{2^n}\times\frac{W}{2^n}}$로 매핑을 하며 이 signal은 meta block에 대한 weight를 생성하는데 공급된다  
- 또한 fixed positional encoding $P^{H, W}\in\mathcal{R}^{2\times H\times W}$를 정의한다  
- 따라서 전체 네트워크는 다음의 식과 같이 정의할 수 있다  
$$F1,...,F_n=b\left(I\vert \theta^b\right)$$  
$$\phi = h\left(F_n\vert \theta^h \right)$$  
$$\theta^{m_i}=w_i\left(\phi\vert\theta^w\right), i=0,...,n$$  
$$S=d\left(\left\{F_i\right\},\left\{P_i\right\}\vert\left\{\theta^{m_i}\right\}\right)$$  

### The encoder and the hypernetwork  
- backbone network는 EfficientNet 기반이며 backbone의 head는 context head로 대체한다  
- feature maps $F_i$를 출력하며 decoder의 size에 맞추기 위해 channel 수를 줄여 $1\times 1$ conv을 적용한다  
- last feature map에서 각 pixel은 input image에서 patches로 인코딩된다  
- 이 patches는 약간의 overlap이 존재하며 limited receptive field로 인해 multiple patches에 걸친 large objects에 대한 성능이 좋지 않을 수 있다  
- 그래서 context head에서 multiple patches에 대한 information을 결합한다  
- context head에서 $2\times 2$, stride 2의 convolution을 적용하며 output channel의 수는 input channel 수의 절반이다  
- 이는 $3\times 3$ convolution 연산보다 더 효율적인 연산량을 가지고 있다  
- 가장 마지막의 feature map은 highest level context를 추출하기 위해 average pooling을 적용하고 upsampling 과정에서 nearest neighbor interpolation 기법을 적용한다  
- 그리고 U-Net 처럼 skip connection을 적용해 해당 라인의 encoder feature map과 concatenate한다  
- weight mapping network $w=\left[w_0, ..., w_n\right]$이 hypernetwork의 핵심이므로 이를 분리해 primary network blocks에 연결하는 것이 더 효율적이다  
- 따라서 h에 바로 연결하는 대신 weight $w_0,...,w_n$은 각 meta blocks에 embedding한다  
- 이에 따른 근거로 context에서 weight로의 매핑이 memory 측면에서 large expansion을 초래해 bottleneck 현상이 일어날 수 있다  

### The decoder (the primary network)  
- decoder에서 각 block은 bilinear upsampling을 따르고 encoder의 feature map과 concatenate한다  
- 전통적인 방식과 달리 hypernetwork에서 decoder의 weight는 input image에 따라 조정될 뿐만 아니라 image의 다양한 영역에 따라 달라진다  
- 그래서 우리는 low level information과 high level information을 효율적으로 결합할 수 있다  
- regular convolution 대신에 우리는 dynamic, patch-wise convolution을 사용한다  
- large model에서는 $4\times 4$보다 작은 patch를 사용하고 smaller model에서는 $8\times 8$보다 작은 patch를 사용한다  
- 각 meta block $m_i$에서 full signal을 사용하는 것은 $\phi$가 많은 weight에 직접 매핑되기 때문에 연산 측면에서도, trainable parameter의 갯수에서도 비효율적이다  
- 따라서 우리는 $\phi$의 channel을 $C_{\phi_0},...,C_{\phi_n}$으로 나눈다  
- channel division은 다음과 같이 정의를 한다  
$$C_{\phi_0},...,C_{\phi_n}=divide_channels\left(C_n, max\left(g_{w_0},...,g_{w_n}\right), \vert\theta^{m_0}\vert,...,\vert\theta^{m_n}\vert\right)$$  

## Experiments  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/HyperSeg/figures/table2.png?raw=true">  

<img src = "https://github.com/Sangh0/Segmentation/blob/main/HyperSeg/figures/table3.png?raw=true">  

<img src = "https://github.com/Sangh0/Segmentation/blob/main/HyperSeg/figures/figure5.png?raw=true">

<img src = "https://github.com/Sangh0/Segmentation/blob/main/HyperSeg/figures/table4.png?raw=true">  